# Analysis with Pandas

**Pandas’** syntax is quite different from SQL. In Pandas, you apply operations on the dataset, and chain them, in order to transform and reshape the data the way you want it.


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 02 23:55:34 2020

@author: PrakharJ

"""

import pandasql as ps
import pandas as pd

airports = pd.read_csv('data/airports.csv')
airport_freq = pd.read_csv('data/airport-frequencies.csv')
runways = pd.read_csv('data/runways.csv')

**SELECT, WHERE, DISTINCT, LIMIT**


Here are same SELECT statements converted to pandas. We sample results with head, and filter them with values. We use unique to remove duplicated results.


In [ ]:
airports

In [ ]:
airports.head(3)

In [ ]:
airports[airports.ident == 'KLAX'].id

In [ ]:
airports.type.unique()

**SELECT with multiple conditions**

We join multiple conditions with an &. If we only want a subset of columns from the table, that subset is applied in another pair of square brackets.

In [ ]:
airports[(airports.iso_region == 'US-CA') & (airports.type == 'seaplane_base')]

In [ ]:
airports[(airports.iso_region == 'US-CA') & (airports.type == 'large_airport')][['ident', 'name', 'municipality']]

***ORDER BY***

By default, Pandas will sort things in ascending order. To reverse that, provide ascending=False.

In [ ]:
airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values('type')

In [ ]:
airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values('type', ascending=False)

**IN… NOT IN**


We know how to filter on a value, but what about a list of values — IN condition? In pandas, .isin() operator works the same way. To negate any condition, use ~.

In [ ]:
airports[airports.type.isin(['heliport', 'balloonport'])]

In [ ]:
airports[~airports.type.isin(['heliport', 'balloonport'])]

**GROUP BY, COUNT, ORDER BY**


Grouping is straightforward: use the .groupby() operator. There’s a subtle difference between semantics of a COUNT in SQL and Pandas. In Pandas, .count() will return the number of non-null/NaN values. To get the same result as the SQL COUNT, use .size().

In [ ]:
airports.groupby(['iso_country', 'type']).size()

In [ ]:
airports.groupby(['iso_country', 'type']).size().to_frame('size').reset_index().sort_values(['iso_country', 'size'], ascending=[True, False])

Above, we group on more than one field. Pandas will sort things on the same list of fields by default, so there’s no need for a .sort_values() in the first example. If we want to use different fields for sorting, or DESC instead of ASC, like in the second example, we have to be explicit:

What is this trickery with .to_frame() and .reset_index()? Because we want to sort by our calculated field (size), this field needs to become part of the DataFrame. After grouping in Pandas, we get back a different type, called a GroupByObject. So we need to convert it back to a DataFrame. With .reset_index(), we restart row numbering for our data frame.

**HAVING**


In SQL, you can additionally filter grouped data using a HAVING condition. In Pandas, you can use .filter() and provide a Python function (or a lambda) that will return True if the group should be included into the result.


In [ ]:
airports[airports.iso_country == 'US'].groupby('type').filter(lambda g: len(g) > 1000).groupby('type').size().sort_values(ascending=False)

**Top N records**


Let’s say we did some preliminary querying, and now have a dataframe called by_country, that contains the number of airports per country:

<img src="head_image.PNG">

In the next example, we order things by airport_count and only select the top 10 countries with the largest count. Second example is the more complicated case, in which we want “the next 10 after the top 10”:

In [ ]:
by_country.nlargest(10, columns='airport_count')

In [ ]:
by_country.nlargest(20, columns='airport_count').tail(10)

***Aggregate functions (MIN, MAX, MEAN)***


Now, given this dataframe or runway data:

<img src="runway_image.PNG">

In [ ]:
runways.agg({'length_ft': ['min', 'max', 'mean', 'median']})

SQL does not have median function. Let’s pretend you wrote a user-defined function to calculate this statistic (since the important part here is syntactic differences between SQL and Pandas).
You will notice that with this SQL query, every statistic is a column. But with this Pandas aggregation, every statistic is a row:

<img src="median_image.PNG">

In [ ]:
Nothing to worry about —simply transpose the dataframe with .T to get columns:

<img src="transpose_image.PNG">

***JOIN***


Use .merge() to join Pandas dataframes.You need to provide which columns to join on (left_on and right_on), and join type: inner (default), left (corresponds to LEFT OUTER in SQL), right (RIGHT OUTER), or outer (FULL OUTER).

In [ ]:
airport_freq.merge(airports[airports.ident == 'KLAX'][['id']], left_on='airport_ref', right_on='id', how='inner')[['airport_ident', 'type', 'description', 'frequency_mhz']]

**UNION ALL and UNION**


Use pd.concat() to UNION ALL two dataframes:

In [ ]:
pd.concat([airports[airports.ident == 'KLAX'][['name', 'municipality']], airports[airports.ident == 'KLGB'][['name', 'municipality']]])

To deduplicate things (equivalent of UNION), you’d also have to add .drop_duplicates().

**INSERT**

So far, we’ve been selecting things, but you may need to modify things as well, in the process of your exploratory analysis. What if you wanted to add some missing records?
There’s no such thing as an INSERT in Pandas. Instead, you would create a new dataframe containing new records, and then concat the two:


In [10]:
df1 = pd.DataFrame({'id': [1, 2], 'name': ['Harry Potter', 'Ron Weasley']})

In [11]:
df2 = pd.DataFrame({'id': [3], 'name': ['Hermione Granger']})

In [ ]:
pd.concat([df1, df2]).reset_index(drop=True)

**UPDATE**


Now we need to fix some bad data in the original dataframe:

In [ ]:
airports.loc[airports['ident'] == 'KLAX', 'home_link'] = 'http://www.lawa.org/welcomelax.aspx'

**DELETE**


The easiest (and the most readable) way to “delete” things from a Pandas dataframe is to subset the dataframe to rows you want to keep. Alternatively, you can get the indices of rows to delete, and .drop() rows using those indices:

In [ ]:
lax_freq = lax_freq[lax_freq.type != 'MISC']

In [ ]:
lax_freq.drop(lax_freq[lax_freq.type == 'MISC'].index)

**Immutability**


I need to mention one important thing — immutability. By default, most operators applied to a Pandas dataframe return a new object. Some operators accept a parameter inplace=True, so you can work with the original dataframe instead. For example, here is how you would reset an index in-place:


In [ ]:
df.reset_index(drop=True, inplace=True)

However, the .loc operator in the UPDATE example above simply locates indices of records to updates, and the values are changed in-place. Also, if you updated all values in a column:

In [ ]:
df['url'] = 'http://google.com'

or added a new calculated column:

In [ ]:
df['total_cost'] = df['price'] * df['quantity']

these things would happen in-place.


# And more!


The nice thing about Pandas is that it’s more than just a query engine. You can do other things with your data, such as:


**Export to a multitude of formats:**

In [ ]:
df.to_csv(...)  # csv file
df.to_hdf(...)  # HDF5 file
df.to_pickle(...)  # serialized object
df.to_sql(...)  # to SQL database
df.to_excel(...)  # to Excel sheet
df.to_json(...)  # to JSON string
df.to_html(...)  # render as HTML table
df.to_feather(...)  # binary feather-format
df.to_latex(...)  # tabular environment table
df.to_stata(...)  # Stata binary data files
df.to_msgpack(...)	# msgpack (serialize) object
df.to_gbq(...)  # to a Google BigQuery table.
df.to_string(...)  # console-friendly tabular output.
df.to_clipboard(...) # clipboard that can be pasted into Excel

**Plot it:**

In [ ]:
top_10.plot(
    x='iso_country', 
    y='airport_count',
    kind='barh',
    figsize=(10, 7),
    title='Top 10 countries with most airports')

to see some really nice charts!

<img src="airports_image.PNG">

Useful Links:

[pandas groupby](https://www.analyticsvidhya.com/blog/2020/03/groupby-pandas-aggregating-data-python/?utm_source=AVLinkedin&utm_medium=post&utm_campaign=2_September_advanced_Article)